In [1]:

from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime as dt
from datetime import timedelta
from selenium.webdriver.chrome.options import Options


# 팝업창 에러 제어
from selenium.common.exceptions import NoAlertPresentException
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import time
import os
import shutil



## 창 닫힘 방지 옵션
options = Options()
options.add_experimental_option("detach", True)

url = "https://tmacs.kotsa.or.kr/web/TG/TG300/TG3100/Tg2127.jsp?mid=S1810"   

driver = webdriver.Chrome(options=options)
driver.get(url)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')


wait = WebDriverWait(driver, 10)


# ---다운로드 폴더에 새로운 파일이 생길때까지 기다리는 함수에 적용하기 위해서 자신의 다운로드 폴더 경로를 하기!
# # 예시: DOWNLOAD_DIR = "C:/Users/YourName/Downloads"
DOWNLOAD_DIR = r"C:\Users\mstel\Downloads"



 

def wait_for_download_complete(download_dir, timeout=30):
    """다운로드 폴더에 새로운 파일이 생길 때까지 대기"""
    before_files = set(os.listdir(download_dir))
    elapsed = 0
    while elapsed < timeout:
        time.sleep(1)
        after_files = set(os.listdir(download_dir))
        new_files = after_files - before_files
        if new_files:
            print(f"✅ 다운로드 완료: {new_files}")
            return list(new_files)[0]  # 새로 생긴 파일 이름 반환
        elapsed += 1
    raise TimeoutError("다운로드가 완료되지 않았습니다.")




def click_detail_button(driver, wait):
    """조회 결과에서 사고현황(세부) 버튼 클릭 → 팝업 열림"""
    detailed_btn = wait.until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "div.btn_box.type02 > a.btn.morebtn"))
    )
    driver.execute_script("arguments[0].scrollIntoView(true);", detailed_btn)
    driver.execute_script("arguments[0].click();", detailed_btn)
    


def handle_popup_download(driver, wait, year_val, sido_text, jijace_text):
    """팝업 창 전환 → 다운로드 → 닫기 → 메인창 복귀"""
    main_window = driver.current_window_handle

    # 팝업으로 전환
    wait.until(lambda d: len(d.window_handles) > 1)
    for handle in driver.window_handles:
        if handle != main_window:
            driver.switch_to.window(handle)
            break

    time.sleep(3)

    # 다운로드 버튼 클릭
    download_btn = wait.until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "div.btn_box.type02.mar_t0 > a"))
    )
    driver.execute_script("arguments[0].click();", download_btn)
    
    # 다운로드 완료 대기
    downloaded_file = wait_for_download_complete(DOWNLOAD_DIR, timeout=60)


    # 팝업 닫기 후 메인으로 복귀
    driver.close()
    driver.switch_to.window(main_window)

    




In [2]:


# 드롭다운 요소 찾기
year_select = Select(driver.find_element(By.ID, "Year"))
years = year_select.options

# 2014~2023 범위만 value로 가져오기
filtered_years = sorted([
    opt.get_attribute("value") 
    for opt in years 
    if 2016<= int(opt.get_attribute("value")) <= 2023
], key=lambda x: int(x))  # 숫자 순 정렬

# 순서대로 선택
for year_value in filtered_years:
    year_select = Select(driver.find_element(By.ID, "Year"))
    year_select.select_by_value(year_value)

    # --- 광역 반복 ---
    sido_select = Select(driver.find_element(By.ID, "sido"))
    sidos = [opt.get_attribute("value") for opt in sido_select.options if opt.get_attribute("value")]

    for sido_val in sidos:
        sido_select = Select(driver.find_element(By.ID, "sido"))
        sido_select.select_by_value(sido_val)
        sido_text = sido_select.first_selected_option.text

        # --- 기초 반복 ---
        wait.until(EC.presence_of_element_located((By.ID, "jijace")))
        jijace_select = Select(driver.find_element(By.ID, "jijace"))
        jijaces = [opt.get_attribute("value") for opt in jijace_select.options if opt.get_attribute("value")]

        for jijace_val in jijaces:
            jijace_select = Select(driver.find_element(By.ID, "jijace"))
            jijace_select.select_by_value(jijace_val)
            jijace_text = jijace_select.first_selected_option.text

            # 조회 버튼 클릭
            download_btn = driver.find_element(By.CSS_SELECTOR, "div.btn_wrap > a")
            download_btn.click()

            ### 에러 팝업 창 처리 ###
            try:
                alert = driver.switch_to.alert
                alert.accept()
            except NoAlertPresentException:
                pass

            #여기에 이제 세부사항 다운받고 복귀하는 함수를 넣을 것임.
            # 2️⃣ 사고현황 버튼 클릭 함수
            click_detail_button(driver, wait)

            # 3️⃣ 팝업에서 다운로드 함수
            handle_popup_download(driver, wait, year_value, sido_text, jijace_text)

            time.sleep(2)  # 안정화 대기

            # 다운로드 대기
            #time.sleep(3)

            # 파일 이동 및 이름 변경
            #latest_file = max([os.path.join(DOWNLOAD_DIR, f) for f in os.listdir(DOWNLOAD_DIR)], key=os.path.getctime)
            #new_name = f"{year_val}_{sido_text}_{jijace_text}.xlsx"
            #shutil.move(latest_file, os.path.join(SAVE_DIR, new_name))

driver.quit()


✅ 다운로드 완료: {'사고다발지점 상세정보.xls'}
✅ 다운로드 완료: {'사고다발지점 상세정보 (1).xls'}
✅ 다운로드 완료: {'사고다발지점 상세정보 (2).xls'}
✅ 다운로드 완료: {'사고다발지점 상세정보 (3).xls'}


InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=138.0.7204.169); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
	GetHandleVerifier [0x0x7ff765d7e415+77285]
	GetHandleVerifier [0x0x7ff765d7e470+77376]
	(No symbol) [0x0x7ff765b49a6a]
	(No symbol) [0x0x7ff765b35cf5]
	(No symbol) [0x0x7ff765b5a7fa]
	(No symbol) [0x0x7ff765bcfc05]
	(No symbol) [0x0x7ff765bf0192]
	(No symbol) [0x0x7ff765bc83e3]
	(No symbol) [0x0x7ff765b91521]
	(No symbol) [0x0x7ff765b922b3]
	GetHandleVerifier [0x0x7ff766061efd+3107021]
	GetHandleVerifier [0x0x7ff76605c29d+3083373]
	GetHandleVerifier [0x0x7ff76607bedd+3213485]
	GetHandleVerifier [0x0x7ff765d9884e+184862]
	GetHandleVerifier [0x0x7ff765da055f+216879]
	GetHandleVerifier [0x0x7ff765d87084+113236]
	GetHandleVerifier [0x0x7ff

In [7]:
import pandas as pd
# ------------- 중복 체크 -------------
df_2023 = pd.read_excel("사고다발지점 상세정보.xls", header=[0,1,2])
df_2023


사고다발지점 상세정보                                                           \
           지자체                     지점명 발생건수(건)                            
            광역   기초 Unnamed: 2_level_2      건수 사망   중상   경상   부상 대형\n사고   
0           서울  강남구           신논현역 사거리      80  1   16   53   10      0   
1           서울  강남구             학동역사거리      79  0   13   57    9      0   
2           서울  강남구    하나저축은행 강남지점(남쪽)      54  0   13   37    4      0   
3           서울  강남구     영동세브란스(강남세브란스)      47  0   19   27    1      0   
4           서울  강남구          도산공원앞 사거리      49  0   14   33    2      0   
5           서울  강남구         역삼역 부근(동쪽)      55  0   22   28    5      0   
6           서울  강남구       삼성제일빌딩 앞 사거리      62  0   16   41    5      0   
7           서울  강남구             수서역사거리      49  2   15   28    4      0   
8           서울  강남구        경복아파트 앞 사거리      38  0   14   23    1      0   
9           서울  강남구            차병원 사거리      48  0   15   32    1      0   
10          서울  강남구           서울세관 사거리      43  0   14   25    4      0   
11          서울  강남구            신사역 오거리      53  0   11   35    7      0   
12          서울  강남구         을지병원 앞 사거리      54  0    7   41    6      0   
13          서울  강남구              뱅뱅사거리      45  0   16   26    3      0   
14          서울  강남구     코엑스교차로(코엑스 북서)      45  0   12   30    3      0   
15          서울  강남구      우성아파트사거리(역삼초)      46  1    7   34    4      0   
16          서울  강남구            선릉역 사거리      46  0   15   27    4      0   
17          서울  강남구        성수대교 남단 사거리      39  0   11   26    2      0   
18          서울  강남구     서울영동우체국 부근(서쪽)      36  1   10   21    4      0   
19          서울  강남구           학여울역 사거리      36  0   13   17    6      0   
20          서울  강남구            휘문고교사거리      36  0   10   20    6      0   
21          서울  강남구          국기원입구 사거리      37  0    8   25    4      0   
22          서울  강남구            신사역 오거리      20  0    7   10    3      0   
23          서울  강남구           신논현역 사거리      21  0    2   17    2      0   
24          서울  강남구           신논현역 사거리      17  0    3   13    1      0   
25          서울  강남구             수서역사거리       8  0    5    2    1      0   
26          서울  강남구            차병원 사거리       9  0    3    5    1      0   
27          서울  강남구             학동역사거리       9  0    0    8    1      0   
28          서울  강남구       삼성제일빌딩 앞 사거리       6  0    0    5    1      0   
29          서울  강남구     영동세브란스(강남세브란스)       5  0    1    4    0      0   
30         NaN  NaN                 합계    1172  5  312  750  105      0   

            ...                                                      \
   사상자수(명)  ...      통합지표                                ACM_PLC_NO   
        사망  ...       다발도        심각도       통합지수 Unnamed: 17_level_2   
0        1  ...  9.366667  13.033333  11.566667        2.016110e+15   
1        0  ...  8.733333  12.466667  10.973333        2.016110e+15   
2        0  ...  6.733333  11.966667   9.873333        2.016110e+15   
3        0  ...  7.133333  12.033333  10.073333        2.016110e+15   
4        0  ...  6.566667  11.466667   9.506667        2.016110e+15   
5        0  ...  7.933333  10.666667   9.573333        2.016110e+15   
6        0  ...  7.833333  10.700000   9.553333        2.016110e+15   
7        2  ...  6.966667   9.100000   8.246667        2.016110e+15   
8        0  ...  5.566667   9.200000   7.746667        2.016110e+15   
9        0  ...  6.700000   9.066667   8.120000        2.016110e+15   
10       0  ...  5.766667   8.933333   7.666667        2.016110e+15   
11       0  ...  6.066667   8.466667   7.506667        2.016110e+15   
12       0  ...  5.733333   8.966667   7.673333        2.016110e+15   
13       0  ...  6.333333   8.466667   7.613333        2.016110e+15   
14       0  ...  5.800000   9.000000   7.720000        2.016110e+15   
15       1  ...  5.366667   7.766667   6.806667        2.016110e+15   
16       0  ...  6.200000   7.500000   6.980000        2.016110e+15   
17       0  ...  5.166667   7.83